In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

DNA_SIZE = 24
POP_SIZE = 200
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.005
N_GENERATIONS = 50
X_BOUND = [-3, 3]
Y_BOUND = [-3, 3]


def F(x, y):
	return 3*(1-x)**2*np.exp(-(x**2)-(y+1)**2)- 10*(x/5 - x**3 - y**5)*np.exp(-x**2-y**2)- 1/3**np.exp(-(x+1)**2 - y**2)

def plot_3d(ax):

	X = np.linspace(*X_BOUND, 100)
	Y = np.linspace(*Y_BOUND, 100)
	X,Y = np.meshgrid(X, Y)
	Z = F(X, Y)
	ax.plot_surface(X,Y,Z,rstride=1,cstride=1,cmap=cm.coolwarm)
	ax.set_zlim(-10,10)
	ax.set_xlabel('x')
	ax.set_ylabel('y')
	ax.set_zlabel('z')
	plt.pause(3)
	plt.show()

def translateDNA(pop): #pop表示种群矩阵，一行表示一个二进制编码表示的DNA，矩阵的行数为种群数目
	x_pop = pop[:,1::2]#奇数列表示X
	y_pop = pop[:,::2] #偶数列表示y
	
	#pop:(POP_SIZE,DNA_SIZE)*(DNA_SIZE,1) --> (POP_SIZE,1)
	x = x_pop.dot(2**np.arange(DNA_SIZE)[::-1])/float(2**DNA_SIZE-1)*(X_BOUND[1]-X_BOUND[0])+X_BOUND[0]
	y = y_pop.dot(2**np.arange(DNA_SIZE)[::-1])/float(2**DNA_SIZE-1)*(Y_BOUND[1]-Y_BOUND[0])+Y_BOUND[0]
	return x,y

def get_fitness(pop): 
    x,y = translateDNA(pop)
    pred = F(x, y)
    return (pred - np.min(pred)) + 1e-3 #减去最小的适应度是为了防止适应度出现负数，通过这一步fitness的范围为[0, np.max(pred)-np.min(pred)],最后在加上一个很小的数防止出现为0的适应度




def crossover_and_mutation(pop, CROSSOVER_RATE = 0.8):
	new_pop = []
	for father in pop:		#遍历种群中的每一个个体，将该个体作为父亲
		child = father		#孩子先得到父亲的全部基因（这里我把一串二进制串的那些0，1称为基因）
		if np.random.rand() < CROSSOVER_RATE:			#产生子代时不是必然发生交叉，而是以一定的概率发生交叉
			mother = pop[np.random.randint(POP_SIZE)]	#再种群中选择另一个个体，并将该个体作为母亲
			cross_points = np.random.randint(low=0, high=DNA_SIZE*2)	#随机产生交叉的点
			child[cross_points:] = mother[cross_points:]		#孩子得到位于交叉点后的母亲的基因
		mutation(child)	#每个后代有一定的机率发生变异
		new_pop.append(child)

	return new_pop

def mutation(child, MUTATION_RATE=0.003):
	if np.random.rand() < MUTATION_RATE: 				#以MUTATION_RATE的概率进行变异
		mutate_point = np.random.randint(0, DNA_SIZE*2)	#随机产生一个实数，代表要变异基因的位置
		child[mutate_point] = child[mutate_point]^1 	#将变异点的二进制为反转

def select(pop, fitness):    # nature selection wrt pop's fitness
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True,
                           p=(fitness)/(fitness.sum()) )
    return pop[idx]

def print_info(pop):
	fitness = get_fitness(pop)
	max_fitness_index = np.argmax(fitness)
	print("max_fitness:", fitness[max_fitness_index])
	x,y = translateDNA(pop)
	print("最优的基因型：", pop[max_fitness_index])
	print("(x, y):", (x[max_fitness_index], y[max_fitness_index]))






In [13]:
pop = np.random.randint(2, size=(POP_SIZE, DNA_SIZE*2)) #matrix (POP_SIZE, DNA_SIZE)
x,y = translateDNA(pop)
pop = np.array(crossover_and_mutation(pop, CROSSOVER_RATE))
fitness = get_fitness(pop)
pop = select(pop, fitness) #选择生成新的种群


array([6.40733269e+00, 6.45901834e+00, 8.36399415e+00, 1.02057263e+01,
       6.44354367e+00, 6.44747646e+00, 6.54138737e+00, 6.85157312e+00,
       5.28012723e+00, 6.98798039e+00, 7.22751388e+00, 6.41080009e+00,
       4.54596524e+00, 8.03579964e+00, 7.86095586e+00, 6.44659348e+00,
       2.36676664e-01, 6.49276864e+00, 6.53037288e+00, 8.11029196e+00,
       7.18186564e+00, 6.24694211e+00, 6.38888572e+00, 6.46288777e+00,
       7.10421721e+00, 6.34924884e+00, 6.41004178e+00, 4.68873824e+00,
       6.44734965e+00, 5.77977782e+00, 5.18929719e+00, 6.73746031e+00,
       6.46231080e+00, 6.30870963e+00, 6.44607127e+00, 1.07299898e+01,
       6.74737388e+00, 6.40079471e+00, 7.76108826e+00, 8.20892553e+00,
       6.51030125e+00, 6.44767674e+00, 4.38462797e+00, 1.40735374e+01,
       6.84973309e+00, 6.44827267e+00, 6.44987645e+00, 7.34035794e+00,
       6.47501567e+00, 5.49693096e+00, 6.64809354e+00, 6.43606267e+00,
       4.56488274e+00, 6.37592486e+00, 9.63660499e+00, 6.99264712e+00,
      

In [3]:
fig = plt.figure()
ax = Axes3D(fig)	
plt.ion()#将画图模式改为交互模式，程序遇到plt.show不会暂停，而是继续执行
plot_3d(ax)

pop = np.random.randint(2, size=(POP_SIZE, DNA_SIZE*2)) #matrix (POP_SIZE, DNA_SIZE)
for _ in range(N_GENERATIONS):#迭代N代
    x,y = translateDNA(pop)
    if 'sca' in locals(): 
        sca.remove()
    sca = ax.scatter(x, y, F(x,y), c='black', marker='o')
    plt.show()
    plt.pause(0.1)
    pop = np.array(crossover_and_mutation(pop, CROSSOVER_RATE))
    #F_values = F(translateDNA(pop)[0], translateDNA(pop)[1])#x, y --> Z matrix
    fitness = get_fitness(pop)
    pop = select(pop, fitness) #选择生成新的种群

print_info(pop)
plt.ioff()
plot_3d(ax)

<Figure size 640x480 with 0 Axes>

max_fitness: 0.001992084018738737
最优的基因型： [1 0 1 1 0 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1
 0 0 1 1 0 0 1 0 0 1 0]
(x, y): (-0.017639459230867516, 1.575408612216032)


In [17]:
pop = np.random.randint(2, size=(POP_SIZE, DNA_SIZE * 2))  # matrix (POP_SIZE, DNA_SIZE)
pop

array([[1, 0, 0, ..., 0, 2, 0],
       [1, 0, 0, ..., 1, 2, 1],
       [1, 1, 1, ..., 0, 1, 0],
       ...,
       [2, 0, 0, ..., 0, 1, 2],
       [2, 2, 1, ..., 2, 1, 0],
       [0, 1, 2, ..., 0, 2, 2]])

In [58]:

a = pop[1,:]
print(a)
print('==')
mutation(a,0.99)
print(a)
print(a[0::6])
print(a[1::6])
print(a[2::6])
print(a[3::6])
print(a[4::6])
print(a[5::6])


[1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0
 1 1 0 1 1 0 0 1 0 1 1]
==
[1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0 0 1 0 0 0 0
 1 1 0 1 1 0 0 1 0 1 1]
[1 0 1 1 1 0 0 0]
[1 0 1 0 0 0 1 0]
[0 0 0 1 1 1 1 1]
[0 0 0 1 1 0 0 0]
[0 1 1 1 1 0 1 1]
[1 1 1 0 0 0 1 1]


In [6]:
plt.figure()
plt.show()

<Figure size 640x480 with 0 Axes>